In [4]:
#written by Noah Friedman (a template for scripts to be excuted in the spyder environment
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util

ImportError: No module named analysis_utils

In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', 275000)

In [4]:
def get_per_case_oncogenic_mut_info(muts):
    oncogenicMuts = muts[muts['oncogenic'].notnull()]
    nMutOncDict = dict(oncogenicMuts['Tumor_Sample_Barcode'].value_counts())
    return nMutOncDict

#TODO flesh this out
def get_per_case_unique_gene_oncogenic_mut_info(muts):
    oncogenicMuts = muts[muts['oncogenic'].notnull()]
    oncogenicMuts['caseGeneAltered'] = oncogenicMuts.apply(lambda row: row['Tumor_Sample_Barcode'] + '_' + row['Hugo_Symbol'], axis=1)
    oncogenicMuts = oncogenicMuts.drop_duplicates(subset=['caseGeneAltered'])
    return dict(oncogenicMuts['Tumor_Sample_Barcode'].value_counts())

In [11]:
def get_per_case_mut_info(nmutDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/nmutInfo_impact_filtered.tsv'):
    df = pd.read_table(nmutDfPath)
    return dict(zip(df['Tumor_Sample_Barcode'], df['Nmut']))

def get_per_case_unique_gene_mut_info(muts):
    muts['caseGeneAltered'] = muts.apply(lambda row: row['Tumor_Sample_Barcode'] + '_' + row['Hugo_Symbol'], axis=1)
    muts = muts.drop_duplicates(subset=['caseGeneAltered'])
    return dict(muts['Tumor_Sample_Barcode'].value_counts())


In [5]:
perCaseOncogenicInfo = get_per_case_oncogenic_mut_info(allImpactMuts)
perCaseUniqueOncogenicInfo = get_per_case_unique_gene_oncogenic_mut_info(allImpactMuts)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
perCaseNmutInfo = get_per_case_mut_info()
perCaseUniqueMutInfo = get_per_case_unique_gene_mut_info(allImpactMuts)

In [15]:
listOfDicts = []
for case, nmut in perCaseNmutInfo.items():
    if nmut != 0:
        nOnc = 0
        nGeneOnc = 0
        if case in perCaseOncogenicInfo: nOnc = perCaseOncogenicInfo[case]
        if case in perCaseUniqueOncogenicInfo: nGeneOnc = perCaseUniqueOncogenicInfo[case]
        listOfDicts.append({
            'fracOnc': 1.0*nOnc/nmut,
            'Nmut': nmut,
            'Tumor_Sample_Barcode': case,
            'RatioGenesOncMutToGenesMut': 1.0*nGeneOnc/perCaseUniqueMutInfo[case]
        })
df = pd.DataFrame(listOfDicts)
        

In [17]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/observedOncFraction.tsv', sep='\t', index=False)

In [1]:
allImpactMuts

NameError: name 'allImpactMuts' is not defined